<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Netflix-prize-problem" data-toc-modified-id="Netflix-prize-problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Netflix prize problem</a></span><ul class="toc-item"><li><span><a href="#loading-data" data-toc-modified-id="loading-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>loading data</a></span></li><li><span><a href="#pre-processing" data-toc-modified-id="pre-processing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>pre-processing</a></span><ul class="toc-item"><li><span><a href="#reorgnize-training" data-toc-modified-id="reorgnize-training-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>reorgnize training</a></span></li><li><span><a href="#reorgnize-probe" data-toc-modified-id="reorgnize-probe-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>reorgnize probe</a></span></li><li><span><a href="#Make-the-Movie_Id-and-User_Id-dense-and-start-from-zero" data-toc-modified-id="Make-the-Movie_Id-and-User_Id-dense-and-start-from-zero-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Make the Movie_Id and User_Id dense and start from zero</a></span></li></ul></li><li><span><a href="#save" data-toc-modified-id="save-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>save</a></span></li></ul></li></ul></div>

# Netflix prize problem

## loading data

In [1]:
import pandas as pd
import numpy as np
import numba

from IPython.display import display
#pd.set_option("display.max_rows", 20)

data_folder = r"D:\projects\NetflixPrizeDataset\archive\\"

def read_movie_titles(file_path):
    
        with open(file_path, mode='rb') as f:

            movie_titles=[]
            for i, line in enumerate(f.readlines()):
                if i+1 == 6483:
                    line = "6483,2001,Smokey Joe's Cafe"
                else:
                    line = line.decode('utf-8')
                line = line.strip()
                line = line.split(',')
                mv = int(line[0])
                year =  -1 if line[1] == 'NULL' else int(line[1])
                title = ",".join(line[2:])
                movie_titles.append((mv, year, title))
            return np.array(movie_titles)
            
movie_titles= read_movie_titles(data_folder + "movie_titles.csv")

In [2]:
display(pd.DataFrame(data=movie_titles).head())

,0,1,2
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [3]:
# thanks for https://www.kaggle.com/laowingkin/netflix-movie-recommendation/notebook
# just help a lot in reading the data

def read_data():
    df1 = pd.read_csv(data_folder + r'combined_data_1.txt',\
                       header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    df = df1
    if True:
        df2 = pd.read_csv(data_folder + r'combined_data_2.txt',\
                           header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
        df = df.append(df2)
    if True:
        df3 = pd.read_csv(data_folder + r'combined_data_3.txt',\
                           header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
        df = df.append(df3)
    if True:
        df4 = pd.read_csv(data_folder + r'combined_data_4.txt',\
                           header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
        df = df.append(df4)

    df = df.reset_index(drop=True)
    return df

df = read_data()
display(df.head())
display(df.tail())
display(df.iloc[::5000000])

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


,Cust_Id,Rating
100498272,1790158,4.0
100498273,1608708,3.0
100498274,234275,1.0
100498275,255278,4.0
100498276,453585,2.0


,Cust_Id,Rating
0,1:,NaN
5000000,2560324,4.0
10000000,2271935,2.0
15000000,1921803,2.0
20000000,1933327,3.0
25000000,1465002,3.0
30000000,961023,4.0
35000000,1372532,5.0
40000000,854274,5.0
45000000,116334,3.0


In [4]:
probe_df = pd.read_csv(data_folder + r'probe.txt',\
                   header = None, names = ['Cust_Id'], usecols = [0])

display(probe_df.head())
display(probe_df.tail())

,Cust_Id
0,1:
1,30878
2,2647871
3,1283744
4,2488120


,Cust_Id
1425328,1288730
1425329,2536567
1425330,1107317
1425331,9999:
1425332,1473765


## pre-processing

### reorgnize training

We have 100,480,507 training samples (include probe dataset)

In [6]:
@numba.jit(nopython=True)
def fill_movies(to_be_fill, index, movie_ids, L):

    assert len(movie_ids) == len(index)
    cnt = 0
    length = len(index)
    for i in range(length-1):
        rep = index[i+1] - index[i] - 1
        assert rep >= 0
        for j in range(rep):
            to_be_fill[cnt] = movie_ids[i]
            cnt += 1

    for i in range(L - index[length-1] - 1):
        to_be_fill[cnt] = movie_ids[length-1]
        cnt += 1
                
    assert cnt == len(to_be_fill)
    
    
def reform_train(df):
    
    df_nan = pd.DataFrame(pd.isnull(df.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    if len(df_nan) == 0:
        return df
    df_nan = df_nan.reset_index()

    movie_np = np.zeros(shape=(len(df) - len(df_nan)), dtype=np.int)
    index = df_nan['index'].values
    L =  len(df)
    movie_ids = np.arange(1, len(df_nan)+1, 1)

    fill_movies(movie_np, index, movie_ids, L)

    
    # remove those Movie ID rows
    df_ = df[pd.notnull(df['Rating'])]

    df_.loc[:,"Movie_Id"] = movie_np.astype(int)
    df_.loc[:,'Cust_Id'] = df_['Cust_Id'].astype(int)
    #print('-Dataset examples-')
    #display(df_)
    return df_

df = reform_train(df)
print("We have %d(%.1f million ) records"%(len(df), len(df)/1E6) ) 

C:\Users\liangh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\liangh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


We have 100480507(100.5 million ) records


### reorgnize probe

we have 1,408,395 samples in probe dataset

In [7]:
def reform_probe(probe_df):

    probe_mv = probe_df[probe_df.Cust_Id.str.contains(":")]
    probe_mv = probe_mv.reset_index()
    
    if len(probe_mv) == 0:
        return probe_df

    probe_movie_np = np.zeros(shape=(len(probe_df) - len(probe_mv)), dtype=np.int)
    index = probe_mv['index'].values
    L =  len(probe_df)
    movie_ids = np.array([int(mv[:-1]) for mv in probe_mv['Cust_Id']])
    
    fill_movies(probe_movie_np, index, movie_ids, len(probe_df))

        
    # remove those Movie ID rows
    probe_df_ = probe_df[~(probe_df.Cust_Id.str.contains(":"))]

    probe_df_.loc[:,'Movie_Id'] = probe_movie_np.astype(int)
    probe_df_.loc[:,'Cust_Id'] = probe_df_['Cust_Id'].astype(int)

    #print('-Dataset examples-')
    #display(probe_df_)
    
    return probe_df_

probe_df = reform_probe(probe_df)
print("We have %d(%.1f million) records for probe"%(len(probe_df), len(probe_df)/1E6) )

We have 1408395(1.4 million) records for probe


### Make the Movie_Id and User_Id dense and start from zero

In [15]:
# remove probe data from training
#print(df.dtypes)
#print(probe_df.dtypes)

def get_index():
    global N, M, IJ_test, V_test, IJ_train, V_train
    
    # I have a good lesson here: https://stackoverflow.com/questions/53645882/pandas-merging-101
    probe_used_df = probe_df.merge(df, on=['Movie_Id','Cust_Id'], how="inner", indicator=False)
    probe_unused_df = probe_df.merge(df, on=['Movie_Id','Cust_Id'], how="left", indicator=True)\
        .query("_merge == 'left_only'").drop('_merge', axis=1)

    #print(len(probe_df))
    #print(len(probe_used_df))
    #print(len(probe_unused_df))

    print("probe records droped:", len(probe_unused_df))
    print("probe records used  :", len(probe_used_df))


    df_wo_probe = df.merge(probe_df, on=['Movie_Id','Cust_Id'], how='left', indicator=True)\
    .query("_merge == 'left_only'").drop('_merge', axis=1)

    J_test_ = probe_used_df["Cust_Id"].values
    I_test = probe_used_df["Movie_Id"].values
    V_test = probe_used_df["Rating"].values

    J_train_ = df_wo_probe["Cust_Id"].values
    I_train = df_wo_probe["Movie_Id"].values
    V_train =df_wo_probe['Rating'].values

    #print(probe_df.dtypes)
    #print(df_wo_probe.dtypes)
    #print(I_test.dtype)
    #print(J_test_.dtype)
    #print(I_train.dtype)
    #print(J_train_.dtype)

    # shift I
    I_test = I_test-1
    I_train = I_train-1


    I_all = np.append(I_test, I_train)
    Iuni = np.unique(I_all)
    print("overall movies", len(Iuni))
    print("  train movies", len(np.unique(I_train)))
    print("   test movies", len(np.unique(I_test)))


    
    # dense J
    J_all = np.append(J_test_, J_train_)
    Juni = np.unique(J_all)
    print("overall users", len(Juni))
    print("  train users", len(np.unique(J_train_)))
    print("   test users", len(np.unique(J_test_)))

    J_train = np.searchsorted(Juni, J_train_).astype(np.int32)
    J_test = np.searchsorted(Juni, J_test_).astype(np.int32)

    #print(I_test.dtype)
    #print(J_test.dtype)
    #print(I_train.dtype)
    #print(J_train.dtype)
    
    print("before: max(User Id) + 1 vs. len(unique users): %d vs. %d"%(max(J_train_)+1, len(Juni)))
    print(" after: max(User Id) + 1 vs. len(unique users): %d vs. %d"%(max(J_train)+1, len(Juni)))

    
    N = len(Iuni)
    M = len(Juni)
    print("Train Movies: %d"%N)
    print("Train  Users: %d"%M)
    
    
    IJ_train = np.c_[I_train, J_train]
    IJ_test = np.c_[I_test, J_test]
    
get_index()

probe records droped: 0
probe records used  : 1408395
overall movies 17770
train movies 17770
test movies 16938
overall users 480189
 train users 480189
  test users 462858
before: max(User Id) + 1 vs. len(unique users): 2649430 vs. 480189
 after: max(User Id) + 1 vs. len(unique users): 480189 vs. 480189
Train Movies: 17770
Train  Users: 480189


##  save

In [17]:
print(IJ_train.dtype)
print(IJ_test.dtype)
print(V_train.dtype)
print(V_test.dtype)

print("N train", len(IJ_train))
print("N test", len(IJ_test))
np.savez_compressed("NetflixPrize.npz", X_train=IJ_train, y_train=V_train,\
                    X_test=IJ_test, y_test=V_test)

int32
int32
float64
float64
N train 99072112
N test 1408395
